In [ ]:
from binance import Client
import pandas as pd

In [ ]:
api_key = ''
api_secret = ''

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
client.get_account()

In [ ]:
pd.DataFrame(client.get_historical_klines('ETHUSDT', '1m', '30 min ago UTC'))

In [ ]:
def get_minute_data(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, '{} ago UTC'.format(lookback)))
    frame = frame.iloc[:, :6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame
   

In [ ]:
test = get_minute_data('ETHUSDT', '1m', '30 min')

In [ ]:
test.Open.plot()

In [ ]:
def strategy_test(symbol, qty, entried=False):
    df = get_minute_data(symbol, '1m', '30 min')
    cumulret = (df.Open.pct_change() +1).cumprod() - 1
    print(cumulret)
    if not entried:
            if cumulret[-1] < -0.002:
                order = client.create_order(symbol=symbol, side='BUY', type='MARKET',
                                           quantity=qty)
                print(order)
                entried = True
            else:
                print('No Trade has been executed')
    if entried:
        while True:
            df = get_minute_data(symbol, '1m', '30 min')
            sincebuy = df.loc[df.index > pd.to_datetime(order['transactTime'], unit='ms')]
            if len(sincebuy) > 0:
                sincebuyret = (sincebuy.Open.pct_change() +1).cumprod() - 1
                if sincebuyret[-1] > 0.0015: 
                    order = client.create_order(symbol=symbol, side='SELL', type='MARKET',
                                           quantity=qty)
                    print(order)
                    break 

In [ ]:
strategy_test('ETHUSDT', 0.002)